### FIRST STEPS
1. sign up for ZGoogle Cloud
2. Create a Project Inside Google Cloud
3. Activate our API Services -> Geocoding API & Places API
4. Get API Key & Restrict
5. pipenv install jupyter requests pandas
6. pipenv run jupyter notebook



In [1]:
import requests
import pandas as pd
from config import GOOGLE_API_KEY
from pprint import pprint
from urllib.parse import urlencode
from urllib.parse import urlencode

### Google Maps Client API

In [2]:
class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = "json"
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API KEY IS REQUIRED")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
            
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            return {}
        latlng = {}
        try:
            latlng = r.json()["results"][0]["geometry"]["location"]
        except:
            pass
        lat,lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng =lng
        return lat, lng
    
    def search(self, keyword="Mexican food", radius=1000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location=location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_endcode = urlencode(params)
        places_url = f"{endpoint}?{params_endcode}"

        r = requests.get(places_url)
        if r.status_code not in range(200, 299):
            return{}
        return r.json()
    
    def detail(self, place_id = "ChIJRbkGvqEgQYYR0r3H3mPvySc"):
        detail_base_endpoint = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id": f"{place_id}",
            "fields": "name,rating,formatted_phone_number,formatted_address,types",
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [3]:
client = GoogleMapsClient(api_key=GOOGLE_API_KEY, address_or_postal_code="Newport Beach, CA")
print(client.lat, client.lng)

33.6188829 -117.9298493


In [37]:
client.search("entertainment", radius=40000, location= "77450")

{'html_attributions': [],
 'next_page_token': 'ATtYBwIq6D7RFDPjrrkUrtPAnldrzePLxXhIrGXWJtqohZceGOJdjfpqP5NsOKBFo_kMU6CgGOlOlREGfNEjH9WkDExSlbk41XPwpC8xWe-Qn5IByo5VWvPUs6J3NFmAj-r5gznnwMERq6zYl1s5r0OAGDfIG2Xm7cFV78GR_Skesa779lqY1ft4Wc_eiRGuNFQmhptC-UPU6dVCHL4YixuCQ589W_cowgnFyMdgUyMW1EG8CTRM-OrcxjSkUiKNfefvUoCFCtcEBLedObg5NJRKnjQ71liUXxTQOiRv6wJLm7L3uyGcZu6VgT2nV7FjmFM25gJzUdeNWGUAu_nC25myJ2rL-sJBZA_Sh2O0EXViQAV2UJknn368ipdXTa39eVcW4Ru4ynJceNMiwRZfly_A0oD3G7hiTxwzpXUVn1LFMSvJJXpUwzOLkiYPq9h0zHgy',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 29.7338599, 'lng': -95.6515817},
    'viewport': {'northeast': {'lat': 29.73526792989272,
      'lng': -95.65028857010728},
     'southwest': {'lat': 29.73256827010727, 'lng': -95.65298822989271}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'name': 'ILLnique Entertainment',
   'place_id': 'ChIJscEFN5neQIYRm8MUID1nFlw',
   'plus_code': {'compound_code': 

In [8]:
client.detail(place_id="ChIJq6qqqrogQYYRQiTtAH3CoSs")

{'html_attributions': [],
 'result': {'formatted_address': '22072 Westheimer Pkwy, Katy, TX 77450, USA',
  'formatted_phone_number': '(281) 398-6000',
  'name': 'Cost Cutters',
  'rating': 3.2,
  'types': ['beauty_salon',
   'hair_care',
   'health',
   'point_of_interest',
   'establishment']},
 'status': 'OK'}